# 📒 S2W7D7: 模拟面试与知识点复盘 (Mock Interview Review)

## 📚 1 基础原理 (The Basics)

### Q1: Softmax 的作用是什么？

**你的回答**:

> Softmax 是激活函数，将任意实数向量转化为一个规范的概率分布。它使得每个输出值都在 0 到 1 之间，并且所有输出值之和为 1。

**👨‍🏫 面试官点评 (标准答案)**:
回答得非常准确。核心关键词是 **归一化 (Normalization)**。

  * **输入**: Logits (比如 `[2.0, 1.0, 0.1]`)，无界，难以直观理解。
  * **输出**: Probabilities (比如 `[0.7, 0.2, 0.1]`)，和为 1。
  * **数学公式**:
    $$\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}$$

<!-- end list -->

```python
import torch
import torch.nn.functional as F

logits = torch.tensor([2.5, -0.1, 0.3])
probs = F.softmax(logits, dim=0)

print(f"Logits: {logits}")
print(f"Probs:  {probs}")
print(f"Sum:    {probs.sum()}") # 结果一定是 1.0
```

### Q2: CrossEntropyLoss (交叉熵损失) 的原理？

**你的回答**:

> (比较模糊，只记得名字)

**👨‍🏫 面试官点评 (标准答案)**:
交叉熵衡量的是 **“预测概率分布” 与 “真实分布 (Label)” 之间的差异**。

  * **直观理解**: 惊讶度/惩罚。如果你对正确答案的预测概率越高，Loss 越小；反之 Loss 越大。
  * **简化公式 (针对 One-Hot)**:
    $$\text{Loss} = - \log(P_{\text{correct}})$$
  * **举例**:
      * 正确类别是 0。
      * 预测概率 $P_0 = 0.8 \rightarrow Loss = -\log(0.8) \approx 0.22$ (惩罚小)
      * 预测概率 $P_0 = 0.1 \rightarrow Loss = -\log(0.1) \approx 2.30$ (惩罚大)

<!-- end list -->

```python
import torch.nn as nn

# 模拟预测：Batch=1, 3分类
logits = torch.tensor([[0.1, 0.8, 0.1]]) # 预测倾向于类别 1
label_correct = torch.tensor([1])        # 真实标签是 1
label_wrong = torch.tensor([0])          # 假设真实标签是 0 (模型猜错了)

criterion = nn.CrossEntropyLoss()

loss_good = criterion(logits, label_correct)
loss_bad = criterion(logits, label_wrong)

print(f"预测准确时的 Loss: {loss_good.item():.4f}") # 应该是 0.22 左右
print(f"预测错误时的 Loss: {loss_bad.item():.4f}")  # 应该是 2.30 左右
```

## 🛠️ 2 模型与工程 (Model & Engineering)

### Q3: BERT 的输入长度限制是多少？如何解决？

**你的回答**:

> 理论最大长度是 512 Token。如果超过会被截断。
> **解决方案**: 分批次输入 (Chunking/Sliding Window)，每次输入小于 512，最后聚合结果。

**👨‍🏫 面试官点评**:
回答正确。BERT 的 Positional Encoding 只有 512 个位置，这是物理硬限制。长文本处理是 NLP 的经典难题。


### Q4: 如何加速模型推理 (Inference Optimization)？

**你的回答**:

> (这是盲区，需要加强记忆)

**👨‍🏫 面试官点评 (加分项)**:
当 Python 推理太慢时，有三种主流方案：

1.  **ONNX / TensorRT (最推荐)**:
      * **原理**: 算子融合 (Operator Fusion) 和 静态图优化。
      * **比喻**: PyTorch 是“边看菜谱边做菜的大厨 (动态图)”，ONNX 是“自动化流水线 (静态图)”，省去了 Python 和 C++ 反复交互的开销。
2.  **量化 (Quantization)**:
      * 从 FP32 (32位浮点) 转为 FP16 或 INT8。体积变小，计算变快。
3.  **知识蒸馏 (Distillation)**:
      * 使用 TinyBERT 或 DistilBERT 替换大模型。

## ⚖️ 第三部分：业务与算法 (Trade-offs & Algorithm)

### Q5: 高风险场景 (如拆弹) 下，Precision 和 Recall 哪个更重要？

**你的回答**:

> Precision (查准率) 更重要。在不确定的时候坚决不能尝试，必须优先提升精确度。

**👨‍🏫 面试官点评**:
非常精准的业务判断。

  * **Weighted Loss 的反思**: 我们项目中为了让机器人动起来，提升了 Recall。但在拆弹/手术场景，我们需要 **High Precision**。
  * **策略**: 此时不应盲目加权 Recall，而应该提高 **置信度阈值 (Threshold)**，例如 `Confidence > 0.99` 才执行。



### Q6: 双指针算法 (LeetCode 16/15) 的时间复杂度？

**你的回答**:

> $O(N^2)$。暴力解法是 $O(N^3)$，双指针省略了一次搜索。

**👨‍🏫 面试官点评**:
正确。

  * **排序**: $O(N \log N)$
  * **双指针搜索**: 固定一个数 $i$，剩下两个数 $L, R$ 在有序数组中通过“两头往中间夹”的方式寻找，将 $O(N^2)$ 的搜索空间降维到了 $O(N)$。
  * **总复杂度**: $O(N) \times O(N) = O(N^2)$。



## 🎓 Stage 2 总结

  * **成就**: 完成了从 BERT 微调到 Weighted Loss 优化，再到独立推理引擎部署的完整闭环。